# 2 Sampling Methods

Learn how to and when to perform the four methods of random sampling: simple, systematic, stratified, and cluster.

# Simple random sampling

The simplest method of sampling a population is the one you've seen already. It is known as simple random sampling (sometimes abbreviated to "SRS"), and involves picking rows at random, one at a time, where each row has the same chance of being picked as any other.

To make it easier to see which rows end up in the sample, it's helpful to include a row ID column in the dataset before you take the sample.

In this chapter, we'll look at sampling methods using a synthetic (fictional) employee attrition dataset from IBM, where "attrition" means leaving the company.

attrition_pop is available; dplyr is loaded.

# Instructions:

- View the attrition_pop dataset. Explore it in the viewer until you are clear on what it contains.
- Set the random seed to a value of your choosing.
- Add a row ID column to the dataset, then use simple random sampling to get 200 rows.
- View the sample dataset, attrition_samp. What do you notice about the row IDs?

In [ ]:
# View the attrition_pop dataset
attrition_pop

# Set the seed
set.seed(123)

attrition_samp <- attrition_pop %>% 
  # Add a row ID column
  rowid_to_column() %>% 
  # Get 200 rows using simple random sampling
  slice_sample(n = 200)

# View the attrition_samp dataset
head(attrition_samp)

# Systematic sampling

One sampling method that avoids randomness is called systematic sampling. Here, you pick rows from the population at regular intervals.

For example, if the population dataset had one thousand rows and you wanted a sample size of five, you'd pick rows 200, 400, 600, 800, and 1000.

attrition_pop is available; dplyr and tibble are loaded.

# Instructions:

- Set the sample size to 200.
- Get the population size from attrition_pop.
- Calculate the interval between rows to be sampled.

In [ ]:
# Set the sample size to 200
sample_size <- 200

# Get the population size from attrition_pop
pop_size <- nrow(attrition_pop)
# Calculate the interval
interval <- pop_size %/% sample_size

- Get the row indexes for the sample as a numeric sequence of interval, 2 * interval, up to sample_size * interval.
- Systematically sample attrition_pop, assigning to attrition_sys_samp.
- Add a row ID column to attrition_pop.
- Get the rows of the population corresponding to row_indexes.

In [ ]:
# From previous step
sample_size <- 200
pop_size <- nrow(attrition_pop)
interval <- pop_size %/% sample_size

# Get row indexes for the sample
row_indexes <- seq_len(sample_size) * interval
attrition_sys_samp <- attrition_pop %>% 
  # Add a row ID column
  rowid_to_column() %>% 
  # Get 200 rows using systematic sampling
  slice(row_indexes)
# See the result
head(attrition_sys_samp)

# Is systematic sampling OK?

Systematic sampling has a problem: if the data has been sorted, or there is some sort of pattern or meaning behind the row order, then the resulting sample may not be representative of the whole population. The problem can be solved by shuffling the rows, but then systematic sampling is equivalent to simple random sampling.

Here you'll look at how to determine whether or not there is a problem.

attrition_sys_samp is available and has been given a row ID column; dplyr and ggplot2 are loaded.

# Instructions:

- Add a row ID column to attrition_pop.
- Using the attrition_pop_id dataset, plot YearsAtCompany versus rowid as a scatter plot, with a smooth trend line.

In [ ]:
# Add a row ID column to attrition_pop
attrition_pop_id <- attrition_pop %>% 
  rowid_to_column()

# Using attrition_pop_id, plot YearsAtCompany vs. rowid
ggplot(attrition_pop_id, aes(rowid, YearsAtCompany)) +
  # Make it a scatter plot
  geom_point() +
  # Add a smooth trend line
  geom_smooth(method = "lm", se = FALSE)

- Shuffle the rows of attrition_pop.
- Add a row ID column to attrition_pop.
- Repeat the plot of YearsAtCompany versus rowid with points and a smooth trend line, this time using attrition_shuffled.

In [ ]:
# Shuffle the rows of attrition_pop then add row IDs
attrition_shuffled <- attrition_pop %>% slice_sample(prop = 1) %>% rowid_to_column()
# Using attrition_shuffled, plot YearsAtCompany vs. rowid
# Add points and a smooth trend line
ggplot(attrition_shuffled, aes(rowid, YearsAtCompany)) +
  geom_point() +
  geom_smooth(method = "lm", se = FALSE)

# Does a systematic sample always produce a sample similar to an simple random sample?

Possible answers

( ) Yes. All sampling (random or non-random) schemes will lead us to similar results.

( ) Yes. We should always expect a representative sample for both systematic and simple random sampling.

(x) No. This only holds if a seed has been set for both processes.

( ) No. This is not true if the data is sorted in some way.

In [ ]:
DM.result = 3

# Proportional stratified sampling

If you are interested in subgroups within the population, then you may need to carefully control the counts of each subgroup within the population. Proportional stratified sampling results in subgroup sizes within the sample that are representative of the subgroup sizes within the population. It is equivalent to performing a simple random sample on each subgroup.

attrition_pop is available; dplyr is loaded.

# Instructions:

- Get the counts of employees by Education level from attrition_pop, sorted by descending count
- Add a percent column of percentages (100 times the count divided by the total count).

In [ ]:
education_counts_pop <- attrition_pop %>% 
  # Count the employees by Education level, sorting by n
  count(Education, sort = TRUE) %>% 
  # Add a percent column
  mutate(percent = 100 * n/sum(n))
# See the results
education_counts_pop

- Use proportional stratified sampling on attrition_pop to get 40% of each Education group. That is, group by Education and perform a simple random sample of proportion 0.4 on each group.
- Ungroup the stratified sample.

In [ ]:
# From previous step
attrition_strat <- attrition_pop %>% 
  group_by(Education) %>% 
  slice_sample(prop = 0.4) %>% 
  ungroup()
# See the result
attrition_strat

- As you did with attrition_pop, get the counts of employees by Education level from attrition_strat, sorted by descending count, then add a percent column of percentages.

In [ ]:
# From previous steps
attrition_pop %>% 
  count(Education, sort = TRUE) %>% 
  mutate(percent = 100 * n / sum(n))
attrition_strat <- attrition_pop %>% 
  group_by(Education) %>% 
  slice_sample(prop = 0.4) %>% 
  ungroup()

# Get the counts and percents from attrition_strat
education_counts_strat <- attrition_strat %>% count(Education, sort =TRUE) %>% mutate(percent = 100 * n/sum(n))
# See the results
education_counts_strat

# Equal counts stratified sampling

If one subgroup is larger than another subgroup in the population, but you don't want to reflect that difference in your analysis, then you can use equal counts stratified sampling to generate samples where each subgroup has the same amount of data. For example, if you are analyzing blood types, O is the most common blood type worldwide, but you may wish to have equal amounts of O, A, B, and AB in your sample.

attrition_pop is available; dplyr is loaded.

# Instructions:

- Use equal counts stratified sampling on attrition_pop to get 30 employees from each Education group. That is, group by Education and perform a simple random sample of size 30 on each group.
- Ungroup the stratified sample.

In [ ]:
# Use equal counts stratified sampling to get 30 employees from each Education group
attrition_eq <- attrition_pop %>% group_by(Education) %>%
slice_sample(n = 30) %>% ungroup()
# See the results
attrition_eq

- Get the counts of employees by Education level from attrition_eq, sorted by descending count.
- Add a percent column of percentages (100 times the count divided by the total count).

In [ ]:
# From previous step
attrition_eq <- attrition_pop %>%
  group_by(Education) %>% 
  slice_sample(n = 30) %>%
  ungroup()

# Get the counts and percents from attrition_eq
education_counts_eq <- attrition_eq %>% 
count(Education, sort =TRUE) %>% mutate(percent = 100 * n/sum(n))
# See the results
education_counts_eq

# Weighted sampling

Stratified sampling provides rules about the probability of picking rows from your dataset at the subgroup level. A generalization of this is weighted sampling, which lets you specify rules about the probability of picking rows at the row level. The probability of picking any given row is proportional to the weight value for that row.

attrition_pop is available; dplyr and ggplot2 are loaded.

# Instructions:

- Using attrition_pop, plot YearsAtCompany as a histogram with a binwidth of 1.

In [ ]:
# Using attrition_pop, plot YearsAtCompany as a histogram with binwidth 1
ggplot(attrition_pop, aes(YearsAtCompany)) + geom_histogram(binwidth = 1)

- Sample 400 employees from attrition_pop weighted by YearsAtCompany.

In [ ]:
# Sample 400 employees weighted by YearsAtCompany
attrition_weight <- attrition_pop %>% 
  slice_sample(n = 400, weight_by = YearsAtCompany)

# See the results
attrition_weight

- Using attrition_weight, plot YearsAtCompany as a histogram with binwidth 1.

In [ ]:
# From previous step
attrition_weight <- attrition_pop %>% 
  slice_sample(n = 400, weight_by = YearsAtCompany)

# Using attrition_weight, plot YearsAtCompany as a histogram with binwidth 1
ggplot(attrition_weight, aes(YearsAtCompany)) + geom_histogram(binwidth = 1)

# Question

Which is higher? The weighted sample mean YearsAtCompany or the population mean YearsAtCompany.

# Possible answers:

(x) Sample mean.

( ) Population mean.

( ) Both means are identical.

( ) It is impossible to calculate the two means.

In [ ]:
DM.result = 1

# Benefits of clustering

Cluster sampling is two-stage sampling technique that is closely related to stratified sampling. First you randomly sample which subgroups to include in the sample, then for each subgroup you randomly sample rows within that group.

In which of the following situations would cluster sampling be preferable to stratified sampling?

# Answer the question:

(x) If the interest is on ensuring each rare group will be represented in the sample selected.

( ) If cost is not a limitation and time can be spent carefully sampling from each group in the population.

( ) If collecting an overall sample requires lots of travel from one group to another to collect samples within each group.

( ) If the focus is on comparing particular subgroups within the population.

# Performing cluster sampling

Now that you know when to use cluster sampling, it's time to put it into action. In this exercise you'll explore the JobRole column of the attrition dataset. You can think of each job role as a subgroup of the whole population of employees.

attrition_pop is available; dplyr is loaded.

# Instructions:

- Get the unique JobRole values from attrition_pop.
- Randomly sample four JobRole values from job_roles_pop.

In [ ]:
# Get unique JobRole values
job_roles_pop <- unique(attrition_pop$JobRole)
# Randomly sample four JobRole values
job_roles_samp <- sample(job_roles_pop, size = 4)
# See the result
job_roles_samp

- Filter attrition_pop for the sampled job roles. That is, filter for rows where JobRole is in job_roles_samp.
- For each job role in the filtered dataset, take a random sample of ten rows.

In [ ]:
# From previous step
job_roles_pop <- unique(attrition_pop$JobRole)
job_roles_samp <- sample(job_roles_pop, size = 4)

# Filter for rows where JobRole is in job_roles_samp
attrition_filtered <- attrition_pop %>% 
  filter(JobRole %in% job_roles_samp)
# Randomly sample 10 employees from each sampled job role
attrition_clus <- attrition_filtered %>% 
  group_by(JobRole) %>% 
  slice_sample(n = 10)
# See the result
attrition_clus

# 3 kinds of sampling

Let's compare the performance of point estimates using simple, stratified, and cluster sampling. Before we do that, you'll have to set up the samples.

In these exercises, we'll use the RelationshipSatisfaction column of the attrition dataset, which categorizes the employee's relationship with the company. It's an ordered factor with four levels: Low, Medium, High, and Very_High.

# Instructions:

- Perform simple random sampling on attrition_pop to get one quarter of the population.

In [ ]:
# Perform simple random sampling to get 0.25 of the population
attrition_srs <- attrition_pop %>% slice_sample(prop = 0.25)

- Perform stratified sampling on attrition_pop to get one quarter of the population of each RelationshipSatisfaction group. Remember to ungroup the result.

In [ ]:
# Perform stratified sampling to get 0.25 of each relationship group
attrition_strat <- attrition_pop %>% 
  group_by(RelationshipSatisfaction) %>% 
  slice_sample(prop = 0.25) %>% 
  ungroup()


- Get unique values of attrition_pop's RelationshipSatisfaction column.
- Randomly sample satisfaction_unique to get two values.
- Perform cluster sampling on the selected satisfaction groups, sampling one quarter of the population and ungrouping the result.

In [ ]:
# Get unique values of RelationshipSatisfaction
satisfaction_unique <- unique(attrition_pop$RelationshipSatisfaction)

# Randomly sample for 2 of the unique satisfaction values
satisfaction_samp <- sample(satisfaction_unique, size = 2)

# Perform cluster sampling on the selected group getting 0.25 of the population
attrition_clust <- attrition_pop %>%
  filter(RelationshipSatisfaction %in% satisfaction_samp) %>% 
  group_by(RelationshipSatisfaction) %>% 
  slice_sample(n = nrow(attrition_pop) / 4) %>% 
  ungroup()

# Summary statistics on different kinds of sample

Now you have three types of sample (simple, stratified, cluster), you can compare point estimates from each sample to the population parameter. That is, you can calculate the same summary statistic on each sample and see how it compares to the summary statistic for the population.

Here, we'll look at how satisfaction with the company affects whether or not the employee leaves the company. That is, you'll calculate the proportion of employees who left the company (they have an Attrition value of "Yes"), for each value of RelationshipSatisfaction.

attrition_pop, attrition_srs, attrition_strat, and attrition_clust are available; dplyr is loaded.

# Instructions:

- Group by RelationshipSatisfaction level.
- Summarize to calculate a column named mean_attrition as the mean of the cases where Attrition is equal to "Yes".

In [ ]:
# Use the whole population dataset 
mean_attrition_pop <- attrition_pop %>% 
  # Group by relationship satisfaction level
  group_by(RelationshipSatisfaction) %>% 
  # Calculate the proportion of employee attrition
  summarize(mean_attrition = mean(Attrition == "Yes"))

# See the result
mean_attrition_pop

- Calculate the proportion of employee attrition for each relationship satisfaction group, this time on the simple random sample, attrition_srs.

In [ ]:
# Calculate the same thing for the simple random sample 
mean_attrition_srs <- attrition_srs %>% 
  group_by(RelationshipSatisfaction) %>% 
  summarize(mean_attrition = mean(Attrition == "Yes"))

# See the result
mean_attrition_srs

- Calculate the proportion of employee attrition for each relationship satisfaction group, this time on the stratified sample, attrition_strat.

In [ ]:
# Calculate the same thing for the stratified sample 
mean_attrition_strat <- attrition_strat %>% 
  group_by(RelationshipSatisfaction) %>% 
  summarize(mean_attrition = mean(Attrition == "Yes"))

# See the result
mean_attrition_strat

- Calculate the proportion of employee attrition for each relationship satisfaction group, this time on the cluster sample, attrition_clust.

In [ ]:
# Calculate the same thing for the cluster sample 
mean_attrition_clust <- attrition_clust %>% 
  group_by(RelationshipSatisfaction) %>% 
  summarize(mean_attrition = mean(Attrition == "Yes"))

# See the result
mean_attrition_clust